In [1]:
import os
os.chdir("../")

In [6]:
import pandas as pd
from src.utils.postgresql_utils import * 
from src.utils.utils import * 

In [48]:
df = pd.read_csv("data/raw/strava_data_24_07_2024.csv")
df.head()

,resource_state,name,distance,moving_time,elapsed_time,total_elevation_gain,type,sport_type,id,start_date,...,from_accepted_tag,pr_count,total_photo_count,has_kudoed,athlete.id,athlete.resource_state,map.id,map.summary_polyline,map.resource_state,workout_type
0,2,Evening Workout,0.0,1871,1871,0.0,Workout,Workout,11964337555,2024-07-23T18:36:13Z,...,False,0,0,False,112963899,1,a11964337555,NaN,2,NaN
1,2,Morning Workout,0.0,1972,1972,0.0,Workout,Workout,11916553016,2024-07-18T06:02:19Z,...,False,0,0,False,112963899,1,a11916553016,NaN,2,NaN
2,2,Evening Walk,2844.7,1988,2023,14.2,Walk,Walk,11907682491,2024-07-16T18:33:47Z,...,False,0,0,False,112963899,1,a11907682491,suj_IivlpABbAH^LFb@JPL^`@JDL?ZMz@i@^M`ASpAGv@U...,2,NaN
3,2,Morning Workout,0.0,1690,1690,0.0,Workout,Workout,11907689470,2024-07-16T07:00:57Z,...,False,0,0,False,112963899,1,a11907689470,NaN,2,NaN
4,2,Evening Walk,3137.3,1563,1563,11.4,Walk,Walk,11872606726,2024-07-12T17:45:32Z,...,False,0,0,False,112963899,1,a11872606726,}si_IiwlpAK{@Co@@MDOGOKu@B{@?[Ks@Cw@DYL]HMJCp@...,2,NaN


In [52]:
df['start_date'] = pd.to_datetime(df['start_date']).dt.tz_localize(None)
df['start_date_local'] = pd.to_datetime(df['start_date_local']).dt.tz_localize(None)

df.head()

,resource_state,name,distance,moving_time,elapsed_time,total_elevation_gain,type,sport_type,id,start_date,...,from_accepted_tag,pr_count,total_photo_count,has_kudoed,athlete.id,athlete.resource_state,map.id,map.summary_polyline,map.resource_state,workout_type
0,2,Evening Workout,0.0,1871,1871,0.0,Workout,Workout,11964337555,2024-07-23 18:36:13,...,False,0,0,False,112963899,1,a11964337555,NaN,2,NaN
1,2,Morning Workout,0.0,1972,1972,0.0,Workout,Workout,11916553016,2024-07-18 06:02:19,...,False,0,0,False,112963899,1,a11916553016,NaN,2,NaN
2,2,Evening Walk,2844.7,1988,2023,14.2,Walk,Walk,11907682491,2024-07-16 18:33:47,...,False,0,0,False,112963899,1,a11907682491,suj_IivlpABbAH^LFb@JPL^`@JDL?ZMz@i@^M`ASpAGv@U...,2,NaN
3,2,Morning Workout,0.0,1690,1690,0.0,Workout,Workout,11907689470,2024-07-16 07:00:57,...,False,0,0,False,112963899,1,a11907689470,NaN,2,NaN
4,2,Evening Walk,3137.3,1563,1563,11.4,Walk,Walk,11872606726,2024-07-12 17:45:32,...,False,0,0,False,112963899,1,a11872606726,}si_IiwlpAK{@Co@@MDOGOKu@B{@?[Ks@Cw@DYL]HMJCp@...,2,NaN


In [53]:
columns_to_rename = {
    'athlete.id': 'athlete_id',
    'athlete.resource_state': 'athlete_resource_state',
    'map.id': 'map_id',
    'map.summary_polyline': 'map_summary_polyline',
    'map.resource_state': 'map_resource_state'
}

df.rename(columns=columns_to_rename, inplace=True)


In [54]:
config = load_config_yaml()
conn = connect_postgresql(config)
cur = conn.cursor()

MySQL connection established!


In [40]:
import psycopg2.extras as extras 

In [56]:
table = 'activities'
tuples = [tuple(x) for x in df.to_numpy()] 
cols = ','.join(list(df.columns)) 

query = "INSERT INTO %s(%s) VALUES %%s" % (table, cols) 

In [57]:
extras.execute_values(cur, query, tuples) 

In [58]:
conn.commit()